# Computational Cognitive Neuroscience

## Assignment 3

#### Douwe van Erp (s4258126) & Arianne Meijer - van de Griend (s4620135)

Recurrent neural networks are able to deal with timeseries data, which is often the case in practical settings. To understand the basics of recurrent neural networks, please consult:
 
- [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- [A Gentle Tutorial of Recurrent Neural Network with Error Backpropagation](https://arxiv.org/pdf/1610.02583.pdf)

In this assignment you will learn to implement a recurrent neural network consisting of 

- [x] one LSTM layer and a 
- [x] linear output layer. 

For training and testing you will use two instances of the create_data dataset which just computes as current output the sum of the two previous inputs.

- [x] Note that you will have to make additional modifications. 
- [ ] define an RNN class, you need to 
- [x] define an iterator which works on sequential data. 
- [ ] replace the Classifier by a Regressor class since you are working with continuous output data. 
- [ ] Also your training loop needs to be adapted.
- [ ] You need to accumulate loss over multiple time steps and you need to reset your network state starting at each epoch.


    1.[ ] Implement an RNN and plot the training and test loss on the toy data as a function of epoch number.
    2.[ ] Run your network for 100 time steps on test data; plot the predicted sum and the actual sum as a function of time.

## TODO
- Fix bug about _children
- Play with network size
- Replace Classifier by Regressor - Because the assignment says so!
- Adapt training loop
- New loss -> Accumulated loss over multiple setps each epoch
- Check plots.

In [5]:
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
import utils
import numpy as np
from chainer.cuda import to_cpu
from chainer.dataset import concat_examples
import matplotlib.pyplot as plt

# Create the data
Given code

In [11]:
def create_data(n=3000):

    X = np.random.rand(n,1).astype('float32')
    T = np.sum(np.hstack((X[0:-1],X[1:])),axis=1)
    T = np.hstack([0, T[0:]]).astype('float32')
    T = T.reshape([n,1])

    return X, T

# Definition Recurrent Neural Network

In [15]:
class RNN(Chain):
    def __init__(self,n_in, n_h1, n_h2, n_out):
        with self.init_scope():
            self.embed = L.EmbedID(n_in, n_h1)
            self.mid = L.LSTM(n_h1, n_h2)  # the LSTM layer
            self.out = L.Linear(n_h2, n_out)  # the feed-forward output layer
            
    def __call__(self, x):
        h1 = self.embed(x)
        h2 = self.mid(h1)
        y = self.out(h2)
        return y

# Definition sequential iterator
From [here](https://docs.chainer.org/en/stable/tutorial/recurrentnet.html)

In [20]:
class ParallelSequentialIterator(chainer.dataset.Iterator):
    def __init__(self, dataset, batch_size, repeat=True):
        self.dataset = dataset
        self.batch_size = batch_size
        self.epoch = 0
        self.is_new_epoch = False
        self.repeat = repeat
        self.offsets = [i * len(dataset) // batch_size for i in range(batch_size)]
        self.iteration = 0

    def __next__(self):
        length = len(self.dataset)
        if not self.repeat and self.iteration * self.batch_size >= length:
            raise StopIteration
        cur_words = self.get_words()
        self.iteration += 1
        next_words = self.get_words()

        epoch = self.iteration * self.batch_size // length
        self.is_new_epoch = self.epoch < epoch
        if self.is_new_epoch:
            self.epoch = epoch

        return list(zip(cur_words, next_words))

    @property
    def epoch_detail(self):
        return self.iteration * self.batch_size / len(self.dataset)

    def get_words(self):
        return [self.dataset[(offset + self.iteration) % len(self.dataset)]
                for offset in self.offsets]

    def serialize(self, serializer):
        self.iteration = serializer('iteration', self.iteration)
        self.epoch = serializer('epoch', self.epoch)

# Training the model

In [7]:
def train_model(model):
    max_epoch = 20
    train_loss = []
    val_loss = []
    
    while train_iter.epoch < max_epoch:
    
        # Next minibatch
        train_batch = train_iter.next()
        image_train, target_train = concat_examples(train_batch)
    
        # Feedforward pass
        prediction_train = model(image_train)
    
        # Softmax cross entropy loss
        loss = F.softmax_cross_entropy(prediction_train, target_train)
    
        # Backpropagation
        model.cleargrads()
        loss.backward()
    
        # Update all the trainable paremters
        optimizer.update()
    
        # Check the validation accuracy of prediction after every epoch
        if train_iter.is_new_epoch:
    
            # Display the training loss
            print('epoch:{:02d} train_loss:{:.04f} '.format(train_iter.epoch, float(to_cpu(loss.data))), '')
    
            train_loss.append(float(to_cpu(loss.data)))
    
            test_losses = []
            test_accuracies = []
            while True:
                test_batch = test_iter.next()
                image_test, target_test = concat_examples(test_batch)
    
                # Forward pass
                prediction_test = model(image_test)
    
                # Calculate the loss
                loss_test = F.softmax_cross_entropy(prediction_test, target_test)
                test_losses.append(to_cpu(loss_test.data))
    
                # Calculate the accuracy
                accuracy = F.accuracy(prediction_test, target_test)
                accuracy.to_cpu()
                test_accuracies.append(accuracy.data)
    
                if test_iter.is_new_epoch:
                    test_iter.epoch = 0
                    test_iter.current_position = 0
                    test_iter.is_new_epoch = False
                    test_iter._pushed_position = None
                    break
    
            print('val_loss:{:.04f} val_accuracy:{:.04f}'.format(np.mean(test_losses), np.mean(test_accuracies)))
            
            val_loss.append(np.mean(test_losses))
            
    return max_epoch, train_loss, val_loss

# Plotting the data

In [8]:
def plot_training(epoch, train_loss, val_loss, title): 
    x = range(epoch)          
    fig = plt.figure()
    ax = plt.subplot(111)
    
    ax.plot(x, val_loss, 'r', label="Validation")
    ax.plot(x, train_loss, 'b', label="Training")
    
    plt.xlabel('Epoch')
    plt.xticks(range(epoch))
    plt.ylabel('Loss')
    plt.title(title)
    ax.legend()
    plt.show()

# Main code

In [12]:
n_epochs = 100
n_train = 3000
n_test = 3000
train_data = create_data(n_train)
test_data = create_data(n_test)

In [26]:
batch_size = 100
train_iter = ParallelSequentialIterator(train_data, batch_size)
test_iter = ParallelSequentialIterator(test_data, 1, repeat=False)

n_in, n_h1, n_h2, n_out = 1000, 100, 50, 1000
model = RNN(n_in, n_h1, n_h2, n_out)
optimizer = optimizers.SGD()
optimizer.setup(model)

epoch, train_loss, val_loss = train_model(model)
plot_training(epoch, train_loss, val_loss, 'RNN accumulated loss per epoch')

AttributeError: 'RNN' object has no attribute '_children'